# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [29]:
# Imports
from robora import Workflow, SQLiteStorageProvider, SessionStorageProvider
from robora.sonar_query import SonarQueryHandler
from library.cyber_responsibility_question import CyberResponsibilityModel, get_question_set
from data import COUNTRIES
import pandas as pd
from pathlib import Path

# Configuration: Set to False to just load existing database
RUN_WORKFLOW = True
EXPORT_PATH = "cyber_responsibility_results.xlsx"

In [30]:
if True:
    storage = SQLiteStorageProvider("cyber_responsibility.db")
else:
    storage = SessionStorageProvider()

# Configure workflow for energy ministries
workflow = Workflow(
    SonarQueryHandler(CyberResponsibilityModel),
    storage,
)

question_set = get_question_set(
    domains=["Energy"],
    countries=COUNTRIES
)

# Limit for testing (remove or increase for full run)
question_set.max_questions = 0

In [31]:
# Run the workflow only if RUN_WORKFLOW is True
if RUN_WORKFLOW:
    await workflow.ask_multiple(question_set, return_results=False)
    print("Workflow execution completed")
else:
    print("Skipping workflow execution - loading existing database")

Found cached response
Using cached response
QueryResponse(full_response={'id': '71476cd9-0295-40ff-b0ff-327aa5908bd8', 'model': 'sonar', 'created': 1759632468, 'usage': {'prompt_tokens': 632, 'completion_tokens': 194, 'total_tokens': 826, 'search_context_size': 'low', 'cost': {'input_tokens_cost': 0.001, 'output_tokens_cost': 0.0, 'request_cost': 0.005, 'total_cost': 0.006}}, 'citations': ['https://generisonline.com/overview-of-cybersecurity-regulations-in-ecuador/', 'https://thegfce.org/member-and-partner/ecuador/', 'https://ncsi.ega.ee/country/ec_2022/?allData=1', 'https://blog.gavindennis.com/cyber-security-in-ecuador/', 'https://www.trade.gov/country-commercial-guides/ecuador-digital-economy', 'https://govinsider.asia/intl-en/article/securing-smart-grid-infrastructure-from-cyber-threats', 'https://academic.oup.com/cybersecurity/article/4/1/tyy002/4990518', 'https://www.csis.org/analysis/why-ecuadors-security-crisis-demands-global-action', 'https://www.oas.org/en/sms/cicte/docs/Nati

In [32]:
answers = [x async for x in workflow.dump_answers()]
df = pd.concat([x.flattened for x in answers])
df

,question,error,domain,country,incarnation_name,responsible_date,responsibility_level,explanation,confidence,enriched_citations
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Dominica,"Ministry of Foreign Affairs, International Bus...",None,ResponsibilityLevel.LOW,The primary government ministry responsible fo...,ConfidenceLevel.HIGH,[{'url': 'https://dominica.gov.dm/government-d...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Belize,"Ministry of Public Utilities, Energy, Logistic...",2012-01-01,ResponsibilityLevel.LOW,The primary energy ministry in Belize is curre...,ConfidenceLevel.HIGH,[{'url': 'https://en.wikipedia.org/wiki/Minist...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Iraq,Ministry of Electricity (Iraq),None,ResponsibilityLevel.LOW,The Ministry of Electricity (MoE) of Iraq is t...,ConfidenceLevel.HIGH,[{'url': 'https://en.wikipedia.org/wiki/Minist...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Moldova,Ministry of Energy of the Republic of Moldova,None,ResponsibilityLevel.MEDIUM,The Ministry of Energy of the Republic of Mold...,ConfidenceLevel.HIGH,[{'url': 'https://energie.gov.md/en/content/se...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Saint Kitts and Nevis,"Ministry of Public Infrastructure, Energy and ...",None,ResponsibilityLevel.LOW,The primary energy ministry in Saint Kitts and...,ConfidenceLevel.HIGH,[{'url': 'https://mopieudt.gov.kn/the-minister...
...,...,...,...,...,...,...,...,...,...,...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Saint Vincent and the Grenadines,"Ministry of Urban Development, Energy, Airport...",2020-11-01,ResponsibilityLevel.LOW,The primary government entity responsible for ...,ConfidenceLevel.MEDIUM,[{'url': 'https://www.thegef.org/projects-oper...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Tunisia,"Ministry of Industry, Energy and Mines (Tunisia)",None,ResponsibilityLevel.LOW,The primary government body responsible for en...,ConfidenceLevel.MEDIUM,[{'url': 'http://mmeipa.africa-eu-energy-partn...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Ecuador,Ministry of Energy and Non-Renewable Natural R...,None,ResponsibilityLevel.NONE,There is no evidence from available sources th...,ConfidenceLevel.HIGH,[{'url': 'https://generisonline.com/overview-o...
0,"Assess the PRIMARY (i.e., most important) Ener...",None,Energy,Mongolia,Ministry of Energy of Mongolia,None,ResponsibilityLevel.NONE,The primary energy ministry in Mongolia is the...,ConfidenceLevel.HIGH,[{'url': 'https://energypedia.info/wiki/Mongol...


In [33]:
df.iloc[1].explanation

"The primary energy ministry in Belize is currently known as the Ministry of Public Utilities, Energy, Logistics and E-Governance (MPUELE), which was formed by integrating several departments including Energy under one ministry. This ministry evolved from the earlier Ministry of Energy, Science & Technology and Public Utilities established in 2012, which had originally absorbed the Geology and Petroleum Department. The Energy Unit within MPUELE handles governance and planning concerning energy, including renewable energy, energy efficiency, and cleaner production. However, none of the sources explicitly attribute responsibility for cybersecurity governance, prevention, planning, response, or enforcement to this ministry or its energy unit. Instead, the ministry focuses on legislative, regulatory, and policy frameworks to secure energy access and infrastructure, public utility services, and digital governance broadly. Given the lack of direct reference to cybersecurity duties related to

In [34]:
# Export results to Excel
import re

# Clean the data to handle special characters
df_export = df.copy()

# Function to clean text for Excel compatibility
def clean_for_excel(text):
    if not isinstance(text, str):
        return text
    # Remove illegal XML characters for Excel
    illegal_chars = re.compile(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]')
    return illegal_chars.sub('', text)

# Clean all object/string columns
for col in df_export.select_dtypes(include=['object']).columns:
    df_export[col] = df_export[col].apply(clean_for_excel)

df_export.to_excel(EXPORT_PATH, index=False, engine='openpyxl')
print(f"Results exported to {EXPORT_PATH}")
print(f"Total rows: {len(df_export)}")

Results exported to cyber_responsibility_results.xlsx
Total rows: 304
